# Pipeclean Test Analysis

Run the cell below. It will prompt you for which directory to access with the pipeclean test results to analyze.

In [1]:
from edfi_perf_test_analysis.pipeclean import run_analysis
run_analysis()

✅ **Running analysis on C:\source\exchange\Suite-3-Performance-Testing\testResults\2022-10-24-14-08-12**

### Exceptions

First five exceptions

❕ **No data to display**

### Failures

...

,Request Count,Failure Count,Avg Response Time,Approx Std Dev
sum,1218.00,0.00,,
mean,,,46.91,1.23
min,,,12.82,-3.33
max,,,5206.65,266.87


### Ten Worst Average Response Times

,Avg Response Time,Request Count,Failure Count,Approx Std Dev,Min Response Time,Max Response Time
Request,,,,,,
GET /data/v3/ed-fi/cohorts,5206.65,1,0,-3.33,5206.65,5206.65
GET /data/v3/ed-fi/studentNeglectedOrDelinquentProgramAssociations,440.57,1,0,-0.28,440.57,440.57
GET /data/v3/ed-fi/studentHomelessProgramAssociations,410.36,1,0,-0.18,410.36,410.36
GET /data/v3/ed-fi/grades,354.76,1,0,-2.38,354.76,354.76
GET /composites/v1/ed-fi/enrollment/sections,324.49,1,0,-2.25,324.49,324.49
GET /composites/v1/ed-fi/enrollment/localEducationAgencies/{id}/schools,308.07,1,0,0.96,308.07,308.07
GET /data/v3/ed-fi/studentEducationOrganizationAssociations,291.15,1,0,-0.57,291.15,291.15
GET /data/v3/ed-fi/studentAssessments,285.15,1,0,2.42,285.15,285.15
GET /data/v3/ed-fi/courseTranscripts,258.43,1,0,0.78,258.43,258.43


### Ten Best Average Response Times

,Avg Response Time,Request Count,Failure Count,Approx Std Dev,Min Response Time,Max Response Time
Request,,,,,,
GET /data/v3/ed-fi/localEducationAgencies/{id},12.82,2,0,1.59,9.49,16.15
GET /data/v3/ed-fi/contactTypeDescriptors/{id},12.90,1,0,0.05,12.90,12.90
GET /data/v3/ed-fi/schoolYearTypes/{id},12.95,1,0,0.03,12.95,12.95
GET /data/v3/ed-fi/accommodationDescriptors/{id},13.04,1,0,-0.02,13.04,13.04
GET /data/v3/ed-fi/academicHonorCategoryDescriptors/{id},13.10,1,0,-0.05,13.10,13.10
GET /data/v3/ed-fi/educationContents/{id},13.31,1,0,-0.15,13.31,13.31
GET /data/v3/ed-fi/absenceEventCategoryDescriptors/{id},13.38,1,0,-0.19,13.38,13.38
GET /data/v3/ed-fi/communityProviderLicenses/{id},13.50,1,0,0.25,13.50,13.50
GET /data/v3/ed-fi/budgets/{id},13.51,1,0,0.24,13.51,13.51
